The EDA and Backward elimination has helped us find the optimal columns for regression training
Now we need to find out what results the machine learning algorithm will give
Here we try XGBoost.

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as stats
from scipy.stats import f_oneway
import matplotlib.pyplot as plt
import xgboost as xg
from pathlib import Path
from openFile import OpenCleanFile

In [5]:
clean = OpenCleanFile()
clean.df_housing.head()
selected_columns = ["COMMERCIAL UNITS", "TOTAL UNITS", "AGE", "TAX CLASS AT PRESENT", "TAX CLASS AT TIME OF SALE", "BOROUGH_NAME"]  
X, y = clean.df_housing[selected_columns], clean.df_housing["SALE PRICE"]
X.head()  


,COMMERCIAL UNITS,TOTAL UNITS,AGE,TAX CLASS AT PRESENT,TAX CLASS AT TIME OF SALE,BOROUGH_NAME
0,0,5,122,2A,2,Manhattan
1,0,10,109,2B,2,Manhattan
2,0,6,122,2A,2,Manhattan
3,0,8,102,2B,2,Manhattan
4,0,24,102,2,2,Manhattan


In [6]:
# create an xgboost regression model
model = XGBRegressor()

NameError: name 'XGBRegressor' is not defined